In [1]:
from keras.models import Sequential, load_model


model = load_model('../Models/train_weights_some_.3dropout.h5')


Using TensorFlow backend.


In [2]:
model.layers

In [19]:
import scipy.misc as sm
import os
import numpy as np

with open('food101/calories.txt') as f:
    calories = {name: float(c) for name, c in map(str.split, f.readlines())}

outdir = 'food101_2'
def get_mini_batch(directory):
    buf = []
    y = []
    count = 0
    for i in sorted(os.listdir(directory)):
        if os.path.splitext(i)[1] =='.jpg':
            if int(os.path.splitext(i)[0]) % 2 == 0:
                full = os.path.join(directory, i)
                img = sm.imread(full)
               # img = sm.imresize(img, (128, 128, 3), 'bilinear')

                #newfile = os.path.join(outdir, i)
                #sm.imsave(newfile, img)

                if img.shape == (128, 128, 3):
                # img -= np.mean(img, axis=0, keepdims=True) 
                   # buf.append(img.reshape((1, ) + img.shape))
                    y.append(calories[full.replace('_2', '')])
    return  np.array(y)



In [2]:
result = []
for x, y in calories.items():
    fname = int(os.path.splitext(os.path.split(x)[1])[0])
    if fname % 2 == 0:
        result.append(float(y))


In [3]:
result = np.array(result)

In [20]:
y = get_mini_batch('food101_2')

In [21]:
y.shape

(50498,)

In [8]:
x = x.reshape(50498, 128 * 128 * 3)
mean = np.mean(x, axis=0)

In [10]:
x -= mean

In [11]:
x /= np.std(x, axis=0)

In [14]:
import scipy.io


In [12]:
import fbpca
pca = fbpca.pca(x, 1000)

In [22]:
scipy.io.savemat('/mnt/temp/usv', { 'u': pca[0], 's': pca[1], 'v':pca[2], 'y': y, 'mean': mean})

In [23]:
scipy.io.savemat('/mnt/temp/y', { 'y': y})

In [8]:
import keras.backend as K
fc7 = K.function([model.layers[0].input], 
                                   [model.layers[10].output], 
                                   allow_input_downcast=True)
fc6 = K.function([model.layers[0].input], 
                                   [model.layers[9].output], 
                                   allow_input_downcast=True)
pool5 = K.function([model.layers[0].input], 
                   [model.layers[8].output], 
                   allow_input_downcast=True)

batch, y = get_mini_batch('pfid/', 101000)
def compute_features(data, func):
    result = []
    for i in range(0, len(data), 100):
        start = i
        end = i + 100
        if end > len(data):
            end = len(data)
        result_fc7 = func([batch[start:end]])[0]
        result.append(result_fc7)
    return np.vstack(result)
# result_fc6 = fc6([batch])
# result_fc5 = pool5([batch])

/home/han/bin/np-mode/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2124: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


In [9]:
p5weights = compute_features(batch, pool5)


In [7]:
feat = get_mini_batch('./food101/',1)

ValueError: all the input arrays must have same number of dimensions

In [10]:
p5weights

array([[ 0.        ,  2.03042626,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.643641  ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.42092323,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  1.87677658,  0.        , ...,  0.        ,
         5.6312499 ,  0.        ],
       [ 0.        ,  0.        ,  0.19676784, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.8047933 , ...,  0.        ,
         2.83738232,  0.        ]], dtype=float32)

In [62]:
fc7weights = compute_features(batch, fc7)
fc6weights = compute_features(batch, fc6)


In [63]:
result_fc7_arr.shape
print(result_fc6.shape)
print(result_fc5.shape)
import scipy.io as si
si.savemat('pfid_data.mat', {'fc7': fc7weights, 
                             'fc6': fc6weights,
                             'pool5': p5weights,
                             'y': y})

(101000, 2048)
(101000, 6400)
